# Basic Summerizer

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from dotenv import load_dotenv
from langchain.docstore.document import Document
load_dotenv()
from langchain.chains.summarize import load_summarize_chain
from typing import List

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id,languages=['en','hi']).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
def text_splitter(transcript: str):
    """
    Splits transcript text into chunks for embeddings.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.create_documents([transcript])

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# ------------------ Structured Schema ------------------
class AnsandTime(BaseModel):
    Summary: List[str] = Field(description="Summarizes the YouTube transcripts  (no timestamps here)")
    timestamps: float = Field(description="The time (in seconds) from where the summary was taken")

structured_model = model.with_structured_output(AnsandTime)
system_prompt = """
You are a concise and efficient summarizer. Your task is to extract the key points from a provided transcript and present them in two ways:

1.  A list of simple bullet points that summarizes the content. Do not include timestamps in this summary.
2.  The exact timestamp (in seconds) of the specific segment of the transcript you are summarizing.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "Transcript:\n{text}")
])


In [4]:
youtube_captions = load_transcript("https://youtu.be/nQa31xdXbGk")
docs =text_splitter(youtube_captions)


In [ ]:
chain = load_summarize_chain(
        llm=structured_model,
        chain_type="map_reduce",   # or "refine", or "stuff" if short
        map_prompt=prompt,
        combine_prompt=prompt
    )
summary_result = chain.run(docs)


C:\Users\prana\AppData\Local\Temp\ipykernel_12712\1609784719.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary_result = chain.run(docs)
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limit

# Summerizer 2

In [18]:
import re
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
        

def parse_transcript(transcript):
    """Extract text and timestamps from transcript"""
    pattern = r'(.*?)\((\d+\.?\d*)\)'
    matches = re.findall(pattern, transcript)
    
    segments = []
    for text, timestamp in matches:
        text = text.strip()
        if text:
            segments.append({
                'text': text,
                'timestamp': float(timestamp)
            })
    
    return segments
def format_for_llm(segments):
    """Format segments for LLM input"""
    formatted = []
    for segment in segments:
        formatted.append(f"[{segment['timestamp']}s] {segment['text']}")
    
    return "\n".join(formatted)

In [ ]:
youtube_captions = load_transcript("https://www.youtube.com/watch?v=LPZh9BOjkQs")
segments = parse_transcript(youtube_captions)
formatted_transcript = format_for_llm(segments)
formatted_transcript

In [28]:
def summarize_video(transcript):
    """Main function to summarize video transcript"""
    
    """# Parse transcript
    segments = parse_transcript(transcript)
    if not segments:
        return "No valid transcript segments found"
    
    # Format for LLM
    formatted_transcript = format_for_llm(segments)
    """
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    
    # Create prompt
    prompt = ChatPromptTemplate.from_template("""
    You are a video summarizer. Analyze this timestamped transcript and provide:

    1. **OVERVIEW**: Brief summary of the video content
    2. **KEY POINTS**: Main points with their timestamps (format: "Point description [timestamp]")
    3. **MAIN TOPICS**: List of topics covered

    Transcript:
    {transcript}

    Format your response clearly with the three sections above.
    Always include timestamps in square brackets for each key point.
    """)
    
    # Generate summary
    response = llm.invoke(prompt.format(transcript=transcript))
    
    return response.content

In [29]:
print(summarize_video(youtube_captions))

Here's a summary of the video content:

---

### 1. OVERVIEW
The video provides a comprehensive explanation of how large language models (LLMs) function, primarily as next-word predictors. It details the two main phases of their training: an initial "pre-training" on vast amounts of text to learn language patterns, followed by "reinforcement learning with human feedback" (RLHF) to align their behavior with user preferences. The explanation also covers the underlying technological advancements, particularly the Transformer architecture and its key components like the attention mechanism, which enable the parallel processing and contextual understanding crucial for modern LLMs, highlighting the immense computational scale involved.

### 2. KEY POINTS
*   Large Language Models (LLMs) are sophisticated mathematical functions that predict what word comes next for any piece of text [33.38].
*   Instead of predicting one word with certainty, LLMs assign a probability to all possible next word

# Summerizer 3 advanced

In [7]:
import re
from typing import List, Dict, Any
from dataclasses import dataclass
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from youtube_transcript_api import YouTubeTranscriptApi
@dataclass
class TimestampedSegment:
    """Represents a segment of transcript with timestamp"""
    text: str
    start_time: float
    end_time: float = None

class SummaryPoint(BaseModel):
    """Individual summary point with timestamp reference"""
    content: str = Field(description="The main point or topic discussed")
    timestamp: float = Field(description="Timestamp in seconds when this topic was discussed")
    importance: str = Field(description="Importance level: high, medium, low")

class BriefSummary(BaseModel):
    key_points: List[SummaryPoint] = Field(description="List of key points with timestamps and also inlcudes brief summary on Each key Point , does not include unimportant topics")

class MainTopic(BaseModel):
    topic: str = Field(description="The topic name")
    timestamp: float = Field(description="Approx timestamp when topic starts")

class VideoSummary(BaseModel):
    """Complete video summary structure"""
    title: str = Field(description="Suggested title for the video based on content")
    overview: str = Field(description="Brief overview of the entire video upto 200 words")
    key_pt_brief_summary : List[BriefSummary] = Field(description="Brief Summary for each Key Point")
    main_topics: List[MainTopic] = Field(description="List of main topics covered")
    duration_summary: str = Field(description="Summary of video duration and pacing")

class YouTubeVideoSummarizer:
    def __init__(self):
       
        self.llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.3
    )
        self.output_parser = PydanticOutputParser(pydantic_object=VideoSummary)
    
    def parse_transcript(self, transcript: str) -> List[TimestampedSegment]:
        """
        Parse transcript text with timestamps in format:
        "text content (timestamp) more text (timestamp)"
        
        Args:
            transcript: Raw transcript string
            
        Returns:
            List of TimestampedSegment objects
        """
        segments = []
        
        # Regular expression to find text and timestamps
        # Pattern: captures text followed by timestamp in parentheses
        pattern = r'(.*?)\((\d+\.?\d*)\)'
        
        matches = re.findall(pattern, transcript)
        
        for i, (text, timestamp) in enumerate(matches):
            text = text.strip()
            if text:  # Only add non-empty text segments
                segment = TimestampedSegment(
                    text=text,
                    start_time=float(timestamp),
                    end_time=float(matches[i+1][1]) if i+1 < len(matches) else None
                )
                segments.append(segment)
        
        return segments
    
    def format_transcript_for_llm(self, segments: List[TimestampedSegment]) -> str:
        """
        Format parsed segments for LLM processing
        
        Args:
            segments: List of timestamped segments
            
        Returns:
            Formatted transcript string
        """
        formatted_parts = []
        
        for segment in segments:
            timestamp_str = f"[{segment.start_time}s]"
            formatted_parts.append(f"{timestamp_str} {segment.text}")
        
        return "\n".join(formatted_parts)
    
    def create_summary_prompt(self) -> ChatPromptTemplate:
        """Create the prompt template for video summarization"""
        
        system_message = SystemMessagePromptTemplate.from_template(
            """You are an expert video content analyzer and summarizer. 
            You will receive a transcript of a YouTube video with timestamps.
            
            Your task is to:
            1. Create a structured summary with key points
            2. Include timestamp references for each point
            3. Identify main topics and themes
            4. Provide an overall assessment of the video content
            
            Be concise but comprehensive. Focus on the most important information.
            Always include the timestamp reference for where you found each piece of information.
            
            {format_instructions}
            """
        )
        
        human_message = HumanMessagePromptTemplate.from_template(
            """Please analyze and summarize the following timestamped video transcript:

            {transcript}
            
            Topics of the Transcript are provided below:
            {topics}
            
            """
        )
        
        return ChatPromptTemplate.from_messages([system_message, human_message])
    
    def summarize_video(self, transcript: str , topics : str) -> Dict[str, Any]:
        """
        Main method to summarize a video transcript
        
        Args:
            transcript: Raw transcript string with timestamps
            
        Returns:
            Structured summary dictionary
        """
        # Parse the transcript
        segments = self.parse_transcript(transcript)
        
        if not segments:
            return {"error": "No valid transcript segments found"}
        
        # Format for LLM
        formatted_transcript = self.format_transcript_for_llm(segments)
        
        # Create prompt
        prompt = self.create_summary_prompt()
        
        # Prepare the input
        formatted_prompt = prompt.format_prompt(
            transcript=formatted_transcript,
            topics = topics,
            format_instructions=self.output_parser.get_format_instructions()
        )
        
        # Generate summary
        try:
            response = self.llm(formatted_prompt.to_messages())
            parsed_output = self.output_parser.parse(response.content)
            
            # Convert to dictionary and add metadata
            summary_dict = parsed_output.dict()
            summary_dict["total_segments"] = len(segments)
            summary_dict["video_duration"] = max([s.start_time for s in segments]) if segments else 0
            
            return response,parsed_output , summary_dict
            
        except Exception as e:
            return {"error": f"Failed to generate summary: {str(e)}"}
    def format_summary_output(self, summary: Dict[str, Any]) -> str:
        if "error" in summary:
            return f"Error: {summary['error']}"
        
        output = []
        output.append(f"📹 VIDEO SUMMARY")
        output.append("=" * 50)
        output.append(f"Title: {summary['title']}")
        output.append(f"Duration: {summary.get('video_duration', 0):.1f} seconds")
        output.append(f"Total Segments: {summary.get('total_segments', 0)}")
        output.append("")
        
        output.append("📋 OVERVIEW:")
        output.append(summary['overview'])
        output.append("")
        
        output.append("🎯 KEY POINTS:")
        for i, point in enumerate(summary['key_pt_brief_summary'], 1):
            output.append(f"{i}. {point['content']}")
            output.append(f"   ⏰ Timestamp: {point['timestamp']}s | Importance: {point['importance']}")
            output.append("")
        
        output.append("📚 MAIN TOPICS:")
        for topic in summary['main_topics']:
            output.append(f"• {topic}")
        output.append("")
        
        output.append("⏱️ PACING ANALYSIS:")
        output.append(summary['duration_summary'])
        
        return "\n".join(output)
    


In [ ]:
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id,languages=['en','hi']).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
sample_transcript =load_transcript("https://youtu.be/sBHeMcxupmE")

# Initialize the summarizer (you'll need to provide your OpenAI API key)
summarizer = YouTubeVideoSummarizer()

# Test transcript parsing
segments = summarizer.parse_transcript(sample_transcript)
print("Parsed Segments:")
for segment in segments:
    print(f"[{segment.start_time}s] {segment.text}")
    break

print("\n" + "="*50)
print("Formatted for LLM:")
formatted = summarizer.format_transcript_for_llm(segments)
print(formatted)

# Generate summary (uncomment when you have API key)
response , parsed_output , summary = summarizer.summarize_video(sample_transcript , topics = topics)


'\n# Initialize the summarizer (you\'ll need to provide your OpenAI API key)\nsummarizer = YouTubeVideoSummarizer()\n\n# Test transcript parsing\nsegments = summarizer.parse_transcript(sample_transcript)\nprint("Parsed Segments:")\nfor segment in segments:\n    print(f"[{segment.start_time}s] {segment.text}")\n    break\n\nprint("\n" + "="*50)\nprint("Formatted for LLM:")\nformatted = summarizer.format_transcript_for_llm(segments)\nprint(formatted)\n\n# Generate summary (uncomment when you have API key)\nresponse , parsed_output , summary = summarizer.summarize_video(sample_transcript , topics = topics)'

In [ ]:
type(response) ,  type(parsed_output) , type(summary)

(langchain_core.messages.ai.AIMessage, __main__.VideoSummary, dict)

In [6]:
def format_summary_output(summary: Dict[str, Any]) -> str:
        if "error" in summary:
            return f"Error: {summary['error']}"
        
        output = []
        output.append(f"📹 VIDEO SUMMARY")
        output.append("=" * 50)
        output.append(f"Title: {summary['title']}")
        output.append(f"Duration: {summary.get('video_duration', 0):.1f} seconds")
        output.append(f"Total Segments: {summary.get('total_segments', 0)}")
        output.append("")
        
        output.append("📚 MAIN TOPICS:")
        for topic in summary['main_topics']:
            output.append(f"• {topic['topic']}\t")
            output.append(f"{topic['timestamp']}")
        output.append("")
        
        output.append("📋 OVERVIEW:")
        output.append(summary['overview'])
        output.append("")
        
        output.append("🎯 KEY POINTS:")
        for i, point in enumerate(summary['key_pt_brief_summary'][0]['key_points'],1):
            output.append(f"{i}. {point['content']}")
            output.append(f"   ⏰ Timestamp: {point['timestamp']}s | Importance: {point['importance']}")
            output.append("")
        
        output.append("⏱️ PACING ANALYSIS:")
        output.append(summary['duration_summary'])
        
        return "\n".join(output)
print(format_summary_output(summary))

📹 VIDEO SUMMARY
Title: Deep Dive into the Anthropic's Model Composition Protocol (MCP) Architecture
Duration: 4627.5 seconds
Total Segments: 1642

📚 MAIN TOPICS:
• Introduction to MCP Architecture	
76.4
• Refining MCP Architecture: Introducing the Client	
349.919
• Refining MCP Architecture: Primitives	
879.12
• Refining MCP Architecture: Data Layer	
1617.84
• Refining MCP Architecture: Transport Layer	
3207.2
• Final MCP Architecture Summary	
4475.199

📋 OVERVIEW:
This video provides a comprehensive explanation of Anthropic's Model Composition Protocol (MCP) architecture.  Starting with a simplified model of a host (AI chatbot) and server, the video progressively introduces the client as a crucial intermediary for communication.  The concept of primitives—tools, resources, and prompts—offered by the server to the host is detailed.  A significant portion focuses on the data layer, explaining JSON RPC 2.0 as its foundation and comparing it to REST APIs.  The video then delves into the t

In [ ]:
embed_url ="https://www.youtube.com/embed/LPZh9BOjkQs"
for x in summary['key_points']:
    timestamp_url_play = f"{embed_url}?start={int(float(x['timestamp']))}&autoplay=1"
    print(timestamp_url_play)

In [88]:
import streamlit as  st
embed_url ="https://www.youtube.com/embed/LPZh9BOjkQs"
for i, point in enumerate(summary['key_points'], 1):
    timestamp_url_play = f"{embed_url}?start={int(float(point['timestamp']))}&autoplay=1"
    print(timestamp_url_play)

https://www.youtube.com/embed/LPZh9BOjkQs?start=33&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=37&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=88&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=108&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=127&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=227&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=254&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=276&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=382&autoplay=1
https://www.youtube.com/embed/LPZh9BOjkQs?start=392&autoplay=1


# Summarier 4 : Summarizer using TOpic and subtopics provided

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import re 
from youtube_transcript_api import YouTubeTranscriptApi
def load_transcript(url: str) -> str | None:
    """
    Fetch transcript for a YouTube video.
    """
    pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11})'
    match = re.search(pattern, url)
    if match:
        video_id = match.group(1)
        try:
            captions = YouTubeTranscriptApi().fetch(video_id,languages=['en','hi']).snippets
            data = [f"{item.text} ({item.start})" for item in captions]
            return " ".join(data)
        except Exception as e:
            print(f"❌ Error fetching transcript: {e}")
            return None
sample_transcript =load_transcript("https://youtu.be/sBHeMcxupmE")
# Initialize Gemini model (Google 2.5 Flash)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.3  # keep it factual
)

# Prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert video summarizer. Your task is to read the provided YouTube video transcript and the list of topics/subtopics, then create a clear, concise, and well-structured summary.
      Highlight the main ideas, key points under each topic, and remove filler or repetitive text. The output should be easy to understand, organized, and suitable for quick learning or revision."""),
    ("human", "Transcript:\n{transcript}\n\nTopics & Subtopics:\n{topics}")
])

# Create chain
summarizer_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

C:\Users\prana\AppData\Local\Temp\ipykernel_24156\2077333188.py:36: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarizer_chain = LLMChain(


In [13]:
import sqlite3

# Open connection to the database file
# If the file doesn't exist, SQLite will create it
conn = sqlite3.connect("ragDatabase.db", check_same_thread=False)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()


In [14]:
thread_id = "test-123"

cursor.execute("SELECT transcript, output_json, created_at FROM transcript_topics WHERE thread_id=?", (thread_id,))
rows = cursor.fetchall()

type(rows[0][1])


str

In [15]:
topics = rows[0][1]  # Assuming this is a JSON string of topics 
summary = summarizer_chain.invoke({
    "transcript": sample_transcript,
    "topics": topics
})


In [17]:
output2 = summary['text']

In [18]:
len(output2) , len(output1)

(13016, 14282)

In [9]:
output1 = """Here's a detailed summary of the YouTube video, structured according to the provided topics and subtopics:

---

### 🎯 Main Topic 1: Introduction to MCP Life Cycle

*   **Subtopic 1.1: Previous Topics Covered**
    The video continues the MCP (Multi-Agent Communication Protocol) playlist. So far, the series has covered the necessity of MCP and its detailed architecture.
*   **Subtopic 1.2: Importance for Future Coding**
    This video introduces the crucial concept of the MCP Life Cycle, explaining how the architecture (host, client, server) works together step-by-step. Understanding this life cycle is vital for future coding, especially when building custom servers and clients, as the code will be based on these concepts.

### 🎯 Main Topic 2: Understanding MCP Life Cycle

*   **Subtopic 2.1: Definition of Session**
    An MCP session is defined as one continuous connection between a client and a server.
*   **Subtopic 2.2: Example of a Session**
    An example provided is a Cloud Desktop (host/client) connected to a GitHub server. When the Cloud Desktop starts, it automatically connects to the GitHub server. This continuous connection, maintained until the Cloud Desktop is closed, constitutes a session.
*   **Subtopic 2.3: Three Stages of MCP Life Cycle**
    The MCP Life Cycle consists of three main stages:
    1.  Initialization
    2.  Normal Operation
    3.  Shutdown
*   **Subtopic 2.4: Brief Overview of Stages**
    *   **Initialization:** The host/client attempts to connect to the server at the start of a session.
    *   **Operation:** User requests are sent to the server, and the server responds.
    *   **Shutdown:** The continuous connection (session) is terminated, typically by closing the host/client.

### 🎯 Main Topic 3: Stage 1: Initialization Phase

*   **Subtopic 3.1: Key Activities in Initialization**
    The initialization phase is the first interaction between the client and server. During this phase, two critical activities occur:
    1.  **Version Compatibility Check:** The client and server check if they are using compatible MCP protocol versions. If versions are incompatible, communication might fail.
    2.  **Capabilities Exchange and Negotiation:** The client informs the server about its capabilities, and the server informs the client about its capabilities. This acts as a "handshake," agreeing on terms for future communication.
*   **Subtopic 3.2: Step 1: Client Sends Initialize Request**
    The client sends an `initialize` JSON RPC request to the server. This request includes:
    *   The client's protocol version (e.g., in date format).
    *   The client's capabilities (e.g., `roots` for directory access, `sampling` for AI assistance).
    *   The client's implementation info (name and version).
*   **Subtopic 3.3: Step 2: Server Responds to Initialize Request**
    Upon receiving the client's request, the server responds with its own information, including:
    *   Its protocol version.
    *   Its capabilities (e.g., `tools` for functions, `resources` for static documents).
    *   Its implementation info (name and version).
*   **Subtopic 3.4: Step 3: Client Sends Initialized Notification**
    Once the client receives a successful response from the server, it sends an `initialized` notification back to the server. This notification has no ID, meaning no response is expected from the server. At this point, the client and server are considered connected for the session.
*   **Subtopic 3.5: Rules During Initialization**
    Two important rules govern the initialization phase:
    1.  The client should not send requests other than `pings` before the server has responded to the `initialize` request.
    2.  The server should not send requests other than `pings` and `logs` before receiving the `initialized` notification.
    These rules ensure a proper handshake and prevent system crashes due to premature communication.
*   **Subtopic 3.6: Practical Demonstration of Initialization**
    The video demonstrates this phase using Cloud Desktop (client) and a local File System server. When Cloud Desktop starts, the terminal logs show:
    *   The client sending an `initialize` request with its protocol version, (empty) capabilities, and implementation info (e.g., "Cloud AI").
    *   The server responding with its protocol version (which might be different), capabilities (e.g., "tools"), and implementation info (e.g., "Secure File System Server").
    *   The client sending an `initialized` notification. This visually confirms the three steps of the initialization phase.

### 🎯 Main Topic 4: Version and Capability Negotiation

*   **Subtopic 4.1: Version Negotiation**
    If the client and server protocol versions (shared during initialization) mismatch, the client checks its configuration file to see if it supports the server's version. If supported, the connection proceeds; otherwise, the client disconnects, and no further communication occurs. In the demo, the connection proceeded, indicating the client supported the server's version.
*   **Subtopic 4.2: Capability Negotiation Overview**
    Capability negotiation establishes which protocol features will be available during the session. It's crucial for setting expectations, allowing the client to know what it can demand from the server and vice-versa.
*   **Subtopic 4.3: Client Capabilities**
    Clients typically offer three major capabilities:
    1.  **Roots:** Allows the client to grant the server access to its base directory (e.g., a project folder), enabling the server to manipulate files within that directory.
    2.  **Sampling:** Enables the server to request AI assistance from the client. For example, a server might ask the client to summarize documents if the client has AI capabilities.
    3.  **Elicitation:** Allows the server to demand incomplete information from the client. For instance, if a client requests GitHub repositories without providing an API key, the server can elicit this missing information.
*   **Subtopic 4.4: Server Capabilities**
    Servers typically offer four main capabilities:
    1.  **Tools:** Provides access to functions the client can use (e.g., `readFile`, `createFile`).
    2.  **Resources:** Allows the client to fetch static documents or data from the server.
    3.  **Prompts:** Helps the client understand how to interact with the server's functionalities.
    4.  **Logging:** Enables the server to send logging statements to the client, providing periodic updates on long-running tasks (e.g., "form filled," "payment done" during a booking process).
*   **Subtopic 4.5: Sub-capabilities**
    Two common sub-capabilities are:
    1.  **List Changed:** The server sends a notification to the client if its list of tools or resources changes (e.g., a new tool is added or removed).
    2.  **Subscribe:** The server sends notifications to the client about changes in a particular subscribed resource (e.g., updates to a `README.md` file in a GitHub repository).

### 🎯 Main Topic 5: Stage 2: Operation Phase

*   **Subtopic 5.1: Rules for Operation Phase**
    During the operation phase, the client and server exchange messages based on the negotiated capabilities. Key rules include:
    1.  Respecting the negotiated protocol version for JSON RPC message framing.
    2.  Only using capabilities that were successfully negotiated during initialization.
*   **Subtopic 5.2: Part 1: Capability Discovery**
    After initialization, the client automatically attempts to discover the exact tools, resources, and prompts the server offers. It sends JSON RPC requests like `tools/list`, `resources/list`, and `prompts/list`. The server then responds with detailed lists of available items, including descriptions and expected arguments for tools.
*   **Subtopic 5.3: Demonstration of Capability Discovery**
    The practical demo showed that immediately after the `initialized` notification, the client sent batch requests for `tools/list`, `prompts/list`, and `resources/list`. The server responded with a detailed list of file system tools (e.g., `readFile`, `writeFile`, `createDirectory`) and `Method Not Found` errors for `prompts/list` and `resources/list` since the File System server didn't support those capabilities.
*   **Subtopic 5.4: Part 2: Tool Calling**
    Once the client knows the available tools, it can call specific tools or access resources based on user requests. This involves sending a `tools/call` JSON RPC request, specifying the tool name and its required arguments. The server then processes the request and sends back the result.
*   **Subtopic 5.5: Demonstration of Tool Calling**
    In the demo, the user asked Cloud Desktop to read the content of "hello.py" on the desktop. The client's interaction with the server involved:
    1.  Calling `listAllowedDirectories` to confirm access to the desktop.
    2.  Calling the `readFile` tool with the path to "hello.py".
    3.  The server responding with the file's content, which Cloud Desktop then displayed to the user.

### 🎯 Main Topic 6: Stage 3: Shutdown Phase

*   **Subtopic 6.1: No JSON RPC Messages**
    A unique characteristic of the shutdown phase is that the client and server do not exchange any JSON RPC messages. The responsibility for terminating the session lies with the transport layer.
*   **Subtopic 6.2: STDIO Transport Shutdown (Local Servers)**
    For local servers using STDIO transport:
    *   **Client-initiated shutdown (common):** The client closes the server's input stream (STDIN). It then waits for the server to exit. If the server doesn't exit, the client sends a low-level `SigTerm` (signal terminate) to the OS to politely request the server to shut down. If that fails, it sends a `SigKill` (signal kill) to forcibly terminate the server.
    *   **Server-initiated shutdown (rare):** The server closes its output stream to the client and exits gracefully.
*   **Subtopic 6.3: HTTP Transport Shutdown (Remote Servers)**
    For remote servers using HTTP transport:
    *   **Client-initiated shutdown (common):** The client simply closes the open HTTP connection to the server.
    *   **Server-initiated shutdown (rare):** The server closes its HTTP connection from its side. In this case, the client should be prepared to handle the dropped connection gracefully, close any running tasks, and attempt to reconnect.
*   **Subtopic 6.4: Practical Demonstration of Shutdown**
    The demo showed that upon closing the Cloud Desktop (client), the terminal logs displayed a "Server Transport Closed" event. Crucially, no JSON RPC messages were observed during this shutdown process, confirming the theoretical explanation.

### 🎯 Main Topic 7: Special Cases in MCP Life Cycle

*   **Subtopic 7.1: Pings**
    *   **Definition:** Pings are lightweight request-response methods defined in MCP.
    *   **Purpose:** To check if the other party (host or server) is still alive and the connection is responsive. They prevent silent connection drops by OS, proxies, or firewalls during periods of inactivity.
    *   **Usage:** Pings can be sent by both client and server. They are useful during initialization to check aliveness before full setup and periodically during long-running tasks to maintain the connection. A ping request includes a method name "ping" and an ID, and the response has the same ID with an empty result.
*   **Subtopic 7.2: Error Handling**
    *   **Purpose:** To signal when something goes wrong with a request. MCP inherits the standard JSON RPC error object structure.
    *   **Common Scenarios:** Errors can occur due to protocol version mismatch during initialization, calling an unsupported method, invalid arguments for a tool, internal server failure, time-outs, or malformed JSON RPC messages.
    *   **Error Object Structure:** A typical error object includes:
        *   `code`: A numerical code indicating the type of error (e.g., -32601 for Method Not Found).
        *   `message`: A human-readable description of the error.
        *   `data` (optional): Additional debugging information.
    *   **Example:** In the demo, when the client asked for `prompts/list` (which the server didn't support), the server responded with an error object containing `code: -32601` and `message: "Method not found"`.
*   **Subtopic 7.3: Time Out**
    *   **Purpose:** To ensure requests don't hang indefinitely, preventing unresponsive servers, freeing up client resources (memory, CPU), and providing timely user feedback.
    *   **How it Works:** The client's MCP SDK allows setting a time-out threshold for requests (e.g., 30 seconds). If the server doesn't respond within this time, the client triggers a time-out. It then sends a `notifications/cancelled` notification to the server, specifying the request ID and the reason (e.g., "Timeout exceeded"). The server, upon receiving this, stops processing the request and does not send a response.
*   **Subtopic 7.4: Progress Notification**
    *   **Purpose:** To provide real-time feedback to the client (and user) about the progress of long-running tasks. This enhances user experience by indicating how much work has been completed and how much is remaining.
    *   **How it Works:** When the client sends a request for a long-running task, it includes a `progress token` in the request's metadata. The server then periodically sends `notifications/progress` messages, referencing this token. These notifications include the current progress (e.g., "60 out of 100") and a descriptive message (e.g., "Searching 600 out of 1000 files"). The client receives these updates and can display them to the user. An example is the progress bar seen in AI research assistants.

### 🎯 Main Topic 8: Conclusion and Next Steps

*   **Subtopic 8.1: Summary of MCP Life Cycle**
    The video comprehensively covered the three phases of the MCP Life Cycle: Initialization, Operation, and Shutdown, along with special cases like Pings, Error Handling, Time Outs, and Progress Notifications.
*   **Subtopic 8.2: Transition to Practical Coding**
    Having covered the theoretical foundations (Why MCP, Architecture, Life Cycle), the series is now ready to move into practical coding. The next videos will focus on building custom MCP servers and clients.

---"""

In [20]:
print(output2)

Here's a summary of the provided YouTube video transcript, structured according to your topics and subtopics:

### MCP Life Cycle Summary

#### Introduction to MCP Life Cycle
The MCP Life Cycle acts as the "rule book" detailing how the MCP architecture (host, client, server) functions together during a session, step-by-step. It covers the entire process from connection establishment to termination.

*   **Definition of MCP Life Cycle**: It describes the complete sequence of steps that govern how a host and a server establish, use, and end a connection during a session.
*   **Definition of Session**: A session is defined as one continuous connection between the client and the server. For example, when Cloud Desktop connects to a GitHub server, a continuous connection is maintained until the desktop is closed.
*   **Three Stages of MCP Life Cycle**: The MCP Life Cycle comprises three main stages:
    1.  **Initialization**: The host/client attempts to connect to the server.
    2.  **Nor